In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm_notebook
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
appid = config['CiNii_web_api']['appid']

In [3]:
for year in tqdm_notebook(range(2017, 2019)):
    KAKEN_ROOT_URL = 'https://kaken.nii.ac.jp/opensearch/?appid=' + appid
    rw = 500

    """ある年度の件数を取得する。"""
    #URL生成
    url = KAKEN_ROOT_URL + '&format=xhtml' + '&s1=' + str(year) + '&s2=' + str(year) + '&o1=1'

    r = requests.get(url)
    html = r.text

    soup = BeautifulSoup(html, "lxml")
    tag = soup.select("p.search-term-selected")
    p = str(tag[0])

    #数字＝件数を抽出する
    pattern = r'([+-]?[0-9]+\,?[0-9]*)'
    numberlist = re.findall(pattern,p)
    cnt = str(numberlist[0])
    cnt = int(cnt.replace(',', ''))
    #cnt
    
    for start in tqdm_notebook(range(1, cnt, rw)):
        """KAKEN API用URLを作成してレスポンスを返す。ファイルを保存する"""
        if start + rw > cnt:
            end = cnt
        else:
            end = start + rw - 1

        url = KAKEN_ROOT_URL + '&format=xml' + '&s1=' + str(year) + '&s2=' + str(year) + '&o1=1' + '&st=' + str(start) + '&rw=' + str(rw)
        r = requests.get(url)
        if r.status_code == 200:
            filename = 'xml/' + str(year) + '_' + str(start) + '-' + str(end) + '.xml'
            with open(filename, mode='w', encoding='utf-8') as f:
                f.write(r.text)

C:\Users\OWNER\Anaconda3\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
